# Språkmodeller for alle 
#### *Et Introduksjonskurs til språkmodeller med Python og Azure OpenAI*

# Introduksjon

Under et prosjekt vi bidro på en stund tilbake fikk vi hands-on erfaring med å bruke språkmodeller (LLMer) i praksis. Vi så hvordan slike modeller kan hjelpe med å spare tid, redusere kostnader og automatisere manuelle oppgaver på en måte som enkelt kan gjenbrukes i andre prosjekter.

I dette kurset ser vi på et konkret og gjenkjennbart case: analyse av resultatene fra en spørreundersøkelse. Dette er en oppgave som kan være vanskelig å strukturere og som kan være ekstremt tidkrevende å gjøre for hånd når det er store volum av tilbakemeldinger. Vi skal lære dere å koble dere til en LLM via API, bruke LangChain Expression Language (LCEL) til å bygge AI-kjeder og hente ut strukturert innsikt med Pydantic, alt i én oversiktlig notebook.


## Installer nødvendige pakker for kurset

In [92]:
import sys

def is_colab():
    try:
        import google.colab
        return True
    except ImportError:
        return False

if is_colab():
    !pip install be-llm101
    try:
        import be_llm101
        print(f"Installed version {be_llm101.__version__} of the course package")
    except ImportError:
        print("Failed to import be-llm101")
else:
    %load_ext autoreload
    %autoreload 2
    sys.path.append("../src")

    print("This notebook is running locally")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
This notebook is running locally


## Sett miljøvariabler

In [95]:
import os

if is_colab():
    from getpass import getpass
    AZURE_OPENAI_API_KEY = getpass("AZURE_OPENAI_API_KEY: ")
    OPENAI_API_VERSION  = getpass("OPENAI_API_VERSION: ")
    AZURE_OPENAI_DEPLOYMENT_O4 =  getpass("AZURE_OPENAI_DEPLOYMENT_O4: ")
    AZURE_OPENAI_DEPLOYMENT_4O =  getpass("AZURE_OPENAI_DEPLOYMENT_4O: ")
    OPENAI_MODEL_O4 =  getpass("OPENAI_MODEL_O4: ")
    OPENAI_MODEL_4O =  getpass("OPENAI_MODEL_4O: ")
    AZURE_OPENAI_ENDPOINT = getpass("AZURE_OPENAI_ENDPOINT: ")

else:
    AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
    OPENAI_API_VERSION = os.getenv("OPENAI_API_VERSION")
    AZURE_OPENAI_DEPLOYMENT_O4 = os.getenv("AZURE_OPENAI_DEPLOYMENT_O4")
    AZURE_OPENAI_DEPLOYMENT_4O = os.getenv("AZURE_OPENAI_DEPLOYMENT_4O")
    OPENAI_MODEL_O4 = os.getenv("OPENAI_MODEL_O4")
    OPENAI_MODEL_4O = os.getenv("OPENAI_MODEL_4O")
    AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")


# Oppgave
Du jobber i et IT-selskap og har fått i oppgave å analysere svarene fra en intern medarbeiderundersøkelse. Undersøkelsen er anonym og du har fått tilsendt en CSV-fil med 50 tilbakemeldinger, én per ansatt. Målet er å finne ut hva folk er fornøyde eller misfornøyde med, og særlig se nærmere på temaene Nettverk, Opplæring og IT-support, som ledelsen har gitt beskjed om at har vært sentrale temaer i undersøkelser de har sendt ut tidligere. Tilbakemeldinger som ikke passer i disse kategoriene skal også få sin plass. Det endelige målet er å lage en oppsummering som kan sendes til ledelsen.

For å jobbe effektivt bruker du en språkmodell til å hjelpe deg med både kategorisering og oppsummering.


## Datasett

Vi har fått rådatasettet levert som en csv-fil hvor vi i utgangspunktet kun er interessert i de to kolonnene "ID" og "Feedback". "ID" (int) er en unik id for hver ansatt og "Feedback" (str) inneholder tilbakemeldingene. 

In [96]:
# Import the raw data containing the feedback. There is one row per employee.

import pandas as pd
import be_llm101
pd.set_option('display.max_colwidth', None) # Ensure no truncated output of dataframe

path_to_survey = be_llm101.get_data_path()
df = pd.read_csv(
    path_to_survey,
    usecols = ["ID", "Feedback"],       # Extract relevant columns
    dtype={"ID": int, "Feedback": str}  # Specify expected types
)

df.head()


,ID,Feedback
0,1,"The network infrastructure is robust and usually efficient, though intermittent glitches sometimes hamper productivity and clarity."
1,2,"Our current infrastructure aligns with business needs, yet occasional rigidity in legacy systems hinders innovative approaches."
2,3,"I appreciate the proactive approach to security, though the multi-step authentication process sometimes seems overly complicated."
3,4,"Our tools maintain high quality with intuitive design and regular updates, even though rare performance lags can hinder productivity."
4,5,"The IT-support team is consistently prompt and courteous, though occasional miscommunications lead to fleeting moments of uncertainty."


Før vi går løs på oppgaven skal du bli bedre kjent med verktøyene og metodene vi skal bruke.

## Koble til Azure OpenAI via LangChain

In [ ]:
# Connect through the API
#!pip install devtools
from langchain_openai import AzureChatOpenAI 
from dotenv import find_dotenv, load_dotenv 
from devtools import pprint # Pretty output


# Get environment variables
load_dotenv(find_dotenv(), override=True)

llm = AzureChatOpenAI(
    azure_endpoint= AZURE_OPENAI_ENDPOINT,
    azure_deployment=AZURE_OPENAI_DEPLOYMENT_4O,
    model=OPENAI_MODEL_4O,
    api_key=AZURE_OPENAI_API_KEY,
    api_version=OPENAI_API_VERSION,
    temperature = 0,
)



### En vanlig LLM-spørring
Enkle LLM-spørringer er bygget opp av noen sentrale deler:

1. Tilkobling til en ressurs, som feks. Azure OpenAI

2. En prompt, som vil si en tekstbasert forespørsel/instruks

3. Sending av prompt til språkmodellen for å hente en respons

#### Test modellen

In [97]:
# Generate the prompt
prompt = 'Hei!'

# Send the prompt and recieve a response
response = llm.invoke(prompt)

# Show the response from the model
pprint(response)


AIMessage(
    content='Hei! Hvordan kan jeg hjelpe deg i dag?',
    additional_kwargs={
        'refusal': None,
    },
    response_metadata={
        'token_usage': {
            'completion_tokens': 11,
            'prompt_tokens': 9,
            'total_tokens': 20,
            'completion_tokens_details': {
                'audio_tokens': 0,
                'reasoning_tokens': 0,
                'accepted_prediction_tokens': 0,
                'rejected_prediction_tokens': 0,
            },
            'prompt_tokens_details': {
                'audio_tokens': 0,
                'cached_tokens': 0,
            },
        },
        'model_name': 'gpt-4o-mini-2024-07-18',
        'system_fingerprint': 'fp_efad92c60b',
        'prompt_filter_results': [
            {
                'prompt_index': 0,
                'content_filter_results': {
                    'hate': {
                        'filtered': False,
                        'severity': 'safe',
                    },


In [98]:
# Show only the content of the response
response.content

'Hei! Hvordan kan jeg hjelpe deg i dag?'

### Temperatur
Som en del av tilkoblingen er det vanlig å oppgi en temperaturparameter. Denne parameteren sier noe om hvor kreativ modellen kan være i responsen sin, og måles numerisk fra 0 og oppover. Hvis parameteren settes til en verdi rundt 0 tillater du liten grad av "randomness", og modellen vil alltid velge ordet med høyest sannsynlighet for å være riktig når den skal generere en respons. Hvis den derimot settes til en verdi nærmere 1 tillater du større grad av kreativitet og "randomness", og modellen kan velge ord som ikke alltid har høyest sannsynlighet for å være riktig i teksten den svarer med. 

#### Lek deg rundt med temperatur-parameteren

In [79]:
# Test out the temperature-parameter

# Define the prompt
prompt = 'Tell me a funny story about a cat. Write max. 5 sentences.'

# Set temperature (0 = most predictable, 1 = most creative and random)
temperature = 0.9  # Try to switch this out with 0.1 and 0.5 to see the difference. 

# Send the prompt and receive a response
response = llm.invoke(prompt, temperature=temperature)

# Print the response from the model
pprint(response.content)

(
    'Once, a mischievous cat named Whiskers decided to take a nap inside a shoe box. When his owner came home, he thou'
    'ght he found a special gift and excitedly unwrapped it, only to be greeted by a startled Whiskers popping out lik'
    'e a surprise party gone wrong. Startled, the owner dropped the box, and Whiskers, thinking it was all a game, lea'
    'ped out and zoomed around the room like he was in the Indy 500. The owner couldn’t stop laughing as Whiskers cras'
    'hed into the curtain, bringing the whole thing down. From that day on, the box was officially designated as Whisk'
    "ers' throne, and he ruled his kingdom with a goofy grin!"
)


## Hvordan lage en enkel AI-kjede med LangChain Expression Language (LCEL)
 
LCEL er en metode for å bygge og kjøre såkalte *kjeder* i LangChain på. Kjeder, eller chains, brukes for å koble sammen ulike AI-komponenter. F.eks. kan språkmodeller, datakilder og logikk kobles sammen til én sammenhengende prosess, dvs. en kjede. LCEL gir et standardisert språk for å definere disse kjedene, og er brukervennlig fordi man slipper å lage alt manuelt med kode. Med andre ord får du en "oppskrift" på hvordan AI-komponentene dine skal jobbe sammen.

**Fordelene med LCEL**

1. Støtter parallell og asynkron kjøring - Ulike deler av kjeden kan kjøre samtidig, og systemet kan behandle flere forespørsler på en gang. Dermed kan oppgaver behandles raskere.
2. Strømming av resultater - Man kan begynne å se svar mens AI-en fremdeles jobber. (Passer spesielt godt for chatbaserte løsninger)
3. Enkel feilsøking - Når kjedene blir komplekse er det viktig å kunne se hva som har blitt gjort underveis. LCEL logger automatisk alt til LangSmith, som gjør det enklere å feilsøke.
4. Standardisert - Alle kjeder i LCEL bruker samme grensesnitt, som gjør dem enkle å kombinere og gjenbruke på tvers av prosjekter. Standardiseringen gjør at du enkelt kan bruke ulike leverandører av språkmodeller (og andre elementer som vektordatabaser, dokument-parsere osv.) uten å måtte tilpasse koden for hvert enkelt API.

LCEL bruker en pipe-operator ```|``` til å koble sammen ulike trinn i kjeden. Den tar ut data fra én komponent og sender den direkte som input til neste komponent. LCEL bruker også ```PromptTemplate```, som kan tenkes på som en mal for teksten du sender til språkmodellen. ```PromptTemplate``` gjør det enkelt å lage dynamiske meldinger ved at man kan sette inn variabler i teksten. Fordelen med dette er at man kan lage én mal, og bruke den med ulike data. Det hjelper deg også med å skille selve teksten fra logikken, og kan gjøre prosessen sikrere ved at man unngår feil som kan oppstå ved manuell string-manipulasjon. Vi skal nå se på noen eksempler med LCEL som bruker pipe-operator og PromptTemplate.
##### Kilder
[LangChain](https://python.langchain.com/docs/concepts/lcel/)

 

### Eksempel med LCEL

In [99]:
from langchain_core.prompts import PromptTemplate
prompt = PromptTemplate.from_template(
    """
    Hi! Please talk like a {role}.
    """
)
pprint(prompt)

PromptTemplate(
    input_variables=['role'],
    input_types={},
    partial_variables={},
    template=(
        '\n'
        '    Hi! Please talk like a {role}.\n'
        '    '
    ),
)


In [100]:
chain = prompt | llm

pprint(
    chain.invoke({"role": "pirate"})
)

AIMessage(
    content="Ahoy, matey! What be ye wantin' to chat about on this fine day upon the high seas? Arrr!",
    additional_kwargs={
        'refusal': None,
    },
    response_metadata={
        'token_usage': {
            'completion_tokens': 28,
            'prompt_tokens': 18,
            'total_tokens': 46,
            'completion_tokens_details': {
                'audio_tokens': 0,
                'reasoning_tokens': 0,
                'accepted_prediction_tokens': 0,
                'rejected_prediction_tokens': 0,
            },
            'prompt_tokens_details': {
                'audio_tokens': 0,
                'cached_tokens': 0,
            },
        },
        'model_name': 'gpt-4o-mini-2024-07-18',
        'system_fingerprint': 'fp_efad92c60b',
        'prompt_filter_results': [
            {
                'prompt_index': 0,
                'content_filter_results': {
                    'hate': {
                        'filtered': False,
                

Som dere ser returnerer modellen mye mer enn kun svaret på prompten vår, nemlig en hel haug med meta-data. Ofte er man kun interessert i tekst-responsen fra modellen og da kan man bruke StrOutputParser for å redusere den unødvendige støyen.

In [101]:
from langchain_core.output_parsers import StrOutputParser
chain_str = chain | StrOutputParser()

chain_str.invoke({"role": "pirate"})


"Ahoy, matey! What be ye wantin' to chat about on this fine day upon the high seas? Arrr!"

Vi vil også introdusere et annet nyttig verktøy, nemlig batching. Så langt har vi brukt 'invoke' for å sende prompten til modellen. Den tar imot en input og returnerer en output. Batch forventer en liste med input og returner en liste med output. Dette er en mye mer effektiv metode når du vet at du skal gjøre flere kall til modellen.

In [59]:
chain_str.batch(
    [
        {"role": "pirate"},
        {"role": "cowboy"},
        {"role": "ninja"},
    ]
)

["Ahoy, matey! What be ye wishin’ to know on this fine day upon the high seas? Speak quickly, for me treasure be waitin'! Arrr! 🌊🏴\u200d☠️",
 "Well howdy there, partner! Ain't nothin' like gatherin' 'round the campfire after a long day's ride. The stars are shinin' bright overhead, and the coyotes are singin' their mournful tunes. What brings ya to this neck of the woods? You lookin' for some fine tales, or maybe just wantin' to chew the fat 'bout the wide-open range? Saddle up, and let’s spin us a yarn! Yeehaw!",
 'Greetings, silent shadow. The wind whispers secrets of the night, and the moon casts a watchful eye. What knowledge do you seek, wise one? In the stillness, we shall uncover the path of the hidden. Choose your words with care, for a ninja’s wisdom is a treasured blade.']

#### Prøv selv - Bygg din egen mal
Finn på en prompt med variabler som f.eks
- Finner antall inbyggere i land X i år Y
 - Finner forventet levealder på hunderase X
 - Gir modellen en rolle X (assistent, detektiv, lege etc) og ber den forklare deg konsept Y
 - Gjør noe helt sykt, her setter kun fantasien din grenser!

Klarer du også å inkorporere batching?

In [103]:
# PromptTemplate as an email-template. 

prompt = PromptTemplate.from_template(
    """
    Your prompt :)
    """
)  

chain_str = prompt | llm | StrOutputParser()

chain_str.invoke({"variabel":...}) # Fill out the blanks!
#chain_str.batch([{"variabel":"..."},{"variabel":"..."}])

'Sure! What would you like to discuss or explore today? You can ask me anything or give me a specific topic to work with!'

# Kategorisering av enkelte tilbakemeldinger

### Bruk LLMen til å kategorisere tilbakemeldingene
I første omgang av kategoriseringen er vi interesserte i å se hvor mange av tilbakemeldingene som passer innenfor de kategoriene ledelsen foreslo, nemlig Network, Training og IT-support. Tilbakemeldinger modellen mener at ikke passer i noen av disse forhåndsbestemte kategoriene vil vi samle opp i en 'Other' kategori.

In [102]:
categorize_prompt = PromptTemplate.from_template(
"""
Categorize the following feedback into one of the following categories:
- Network
- Training
- IT-support
- Other

Feedback:
<feedback>
{feedback}
</feedback>
"""
)

categorize_chain = categorize_prompt | llm | StrOutputParser()

categorize_chain.invoke({"feedback": "I am very happy with the IT support I received last week."})

'Category: IT-support'

# Datavalidering og strukturering med Pydantic

Når vi jobber med store språkmodeller (LLMs), er det viktig å kunne strukturere og validere output. Vi ønsker ikke bare et svar, men et svar på riktig format.
Derfor bruker vi verktøy som Pydantic, som hjelper oss med:
- Å strukturere data på en tydelig og eksplisitt måte
- Å validere at data har riktig type og verdi
- Å tvinge output fra LLM til å følge en bestemt mal (for eksempel en JSON-struktur)

## Hva er Pydantic?
Pydantic er et Python-bibliotek for datavalidering basert på Python type hints. Hovedklassen er BaseModel som vi arver fra når vi definerer vår egen datastruktur.
Pydantic gir oss:
1. Datavalidering
    - Du kan begrense hva som er lovlige verdier. F.eks: Godkjente land = ["Norge", "Sverige", "Finland"] → Australia blir avvist.
2. Typekonvertering
    - Hvis du sier at du forventer en int og sender inn "1" (str), vil den konverteres automatisk.
3. Strukturell kontroll
    - Du kan bruke Pydantic-modellen til å validere at output fra LLM matcher en bestemt struktur, som JSON.

### Eksempel: Enkel modell med ett felt

In [104]:
from pydantic import BaseModel, Field

class Categorize(BaseModel):
    "Categorization of a single feedback entry from an IT survey."
    category : str = Field(description="The best fitting category. Only one.") 


Her har vi definert en modell med ett felt: category.
Modellen krever at category er en streng og vil derfor avvise andre typer. Dette blir for eksempel ulovlig

In [105]:
Categorize(category=1)

ValidationError: 1 validation error for Categorize
category
  Input should be a valid string [type=string_type, input_value=1, input_type=int]
    For further information visit https://errors.pydantic.dev/2.9/v/string_type

Mens dette er helt ok

In [106]:
Categorize(category="Network")

Categorize(category='Network')

```Field()``` kan brukes til å spesifisere beskrivelse, standardverdi, valideringsregler og mer. Vi bruker det her for å gi modellen litt mer kontekst.
De spesielt interesserte kan lese mer om funksjonaliteten [her](https://docs.pydantic.dev/latest/concepts/fields/).

## Bruke modellen sammen med LLM
Vi bruker en wrapperen ```with_structured_output(...)``` for å fortelle LLMen at den må svare med output som matcher Pydantic-modellen.
### Eksempel

In [107]:
categorize_chain_structured_output = categorize_prompt | llm.with_structured_output(
    Categorize,
    method="json_schema", # Demands JSON-schema for output
    strict=True           # The model must adhere to the specified schema, no extra fields or missing fields are allowed. All types must be an exact match. 
)

categorize_chain_structured_output.invoke(
    {"feedback": "I am very happy with the IT support I received last week."}
)

Categorize(category='IT-support')

## Begrense mulige verdier med Literal
Noen ganger ønsker vi å begrense hvilke verdier som er gyldige, slik som å si at category bare kan være én av fire forhåndsdefinerte valg.
Dette gjør vi med ```Literal```.

In [108]:
from typing import Literal

CATEGORIES = Literal[
    "Network",
    "Training",
    "IT-support",
    'Other'
]

Nå kan vi bruke ```CATEGORIES``` som en type i Pydantic-modellen vår. Da vil modellen validere at det bare kommer svar som matcher én av disse fire.

### Eksempel: Strukturert output med begrensede kategorier

In [109]:
from typing import Literal

categorize_prompt = PromptTemplate.from_template(            #Notice that we no longer pass the list of categories with the prompt.
"""
Categorize the following feedback into the provided categories. 

Feedback:
<feedback>
{feedback}
</feedback>
"""
)



class CategorizeFromOptions(BaseModel):
    "Categorization of a single feedback entry from an IT survey."
    category: CATEGORIES = Field(
        description="Chosen category for the feedback. Choose 'Other' if the other categories provided are not a good fit."  
    )


categorize_chain_structured_output = categorize_prompt | llm.with_structured_output(
    CategorizeFromOptions,
    method="json_schema",
    strict=True,
)

In [110]:
result = categorize_chain_structured_output.invoke(
    {"feedback": "I am very happy with the IT support I received last week."}
)

result

CategorizeFromOptions(category='IT-support')

Nå har vi satt opp en modell hvor vi er sikre på:
- At outputen er i riktig format
- At category bare kan være en av de forhåndsdefinerte verdiene

Dette er utrolig nyttig hvis du skal bruke output videre i en ny prompt, i en database eller som input til et system.

# Prøv selv
Nå kan du øve deg på å jobbe med pydantic som en del av LCEL-kjeden. Prøv å 

1. Lag en literal med nye kategorier
2. Inkorporer den i en klasse som bruker BaseModel
3. Formuler en prompt som passer en eller flere av kategoriene
4. Lag en LCEL-kjede
5. Kall på modellen og print resultatet

Ble det som forventet? Her kan du utfordre deg selv på nytt med å inkorporere batching, gir forskjellige prompts de forventede resultatene? Varierer modellen mye i svarene sine? Prøv deg frem!




In [ ]:
TEST_CATEGORIES = Literal[
    #.....
]

class TestCategorize(BaseModel):
    #.....


test_prompt = PromptTemplate.from_template(            #Notice that we no longer pass the list of categories with the prompt.
    #......
)


test_categorize_chain = test_prompt | llm.with_structured_output(
  #......
)

result = test_categorize_chain.invoke(
    {"feedback": "My cat hates tuna."}
)

result

## Chain-of-thought
 
Chain of Thought (CoT) er en teknikk innen prompt engineering som hjelper språkmodeller med å løse oppgaver som krever flere tankesteg. I stedet for å hoppe rett til svaret, blir modellen ledet gjennom en logisk og trinnvis prosess som kan gi mer presise og gjennomtenkte svar [1].
 
Du kan altså be modellen om å "tenke høyt" under oppgaven og forklare stegene sine før den leverer et endelig svar. Dette ber du om i prompten som sendes inn.
 
Eksempel på en prompt **uten** CoT:
 
    Prompt: "Hvor mange armer har Eline og Kaspara?"
 
    Svar: "4"
 
Eksempel på en prompt **med** CoT:
 
    Prompt: "Hvor mange armer har Eline og Kaspara? Tenk trinn for trinn."
 
    Svar: "En person har to armer. To personer betyr 2x2 = 4 armer. Svaret er 4."
 
Det kan være fordelaktig å bruke CoT når man jobber med komplekse oppgaver, da nøyaktigheten på outputet fra modellen øker når den "får lov" til å jobbe seg gjennom problemet.
 
I tillegg kan du se hvordan modellen tenker, som gjør det lettere for deg å evaluere svaret. Det blir også lettere å se hvor det gikk galt hvis modellen svarer feil.

OBS: Det er viktig å huske at man ikke har fått fasit selv om man kan se hva modellen "tenker"!
 
##### *Kilder*
[1] [IBM](https://www.ibm.com/think/topics/chain-of-thoughts)

## Eksempel: CoT vs. non-CoT

In [111]:
# Vi minner om
categorize_prompt = PromptTemplate.from_template(            #Notice that we no longer pass the list of categories with the prompt.
"""
Categorize the following feedback into the provided categories. 

Feedback:
<feedback>
{feedback}
</feedback>
"""
)

# CoT
class CategorizeCot(BaseModel):
    "Categorization of a single feedback entry from an IT survey."
    chain_of_thought: str = Field(
        description="Use this space to think through the categorization."  # We have defined a variable that explicitly asks the model to think in chain-of-thought
    )
    category: CATEGORIES = Field(
        description="Chosen category for the feedback. Choose 'Other' if the other categories provided are not a good fit."  
    )


categorize_chain_cot = categorize_prompt | llm.with_structured_output(
    CategorizeCot,
    method="json_schema",
    strict=True,
)

In [112]:
result = categorize_chain_cot.invoke("The internet-speed is so slow I could chase after the IT-survice guy with a baseball-bat!")
print(result.chain_of_thought) 
print(result.category)

The feedback specifically mentions issues with internet speed, which falls under the category of Network. The frustration expressed indicates a problem with the network performance rather than IT support or training.
Network


# Resonneringsmodell

Resonneringsmodeller, som Azure Open AI sin O4-mini-modell, er språkmodeller som er spesielt trent på å tenke før de svarer. Altså *resonnerer* de seg frem til et svar, i motsetning til en typisk LLM som gir raske svar.
Resonneringsmodeller bruker chain of thought til å bryte ned oppgaven i mindre deler, for å så jobbe seg gjennom problemstillingen stegvis [3]. Dette er fordelaktig å bruke til oppgaver som krever kompleks problemløsning, logisk tenkning som koding eller matematikk eller til oppgaver med flere steg. Det vil også være fordelaktig å bruke i situasjoner der nøyaktighet og forklarbarhet er viktig [2].
 
 
##### Kilder
[OpenAI](https://platform.openai.com/docs/guides/reasoning?api-mode=chat)

[AIavisen](https://aiavisen.no/resonneringsmodeller/)

In [118]:
# Initialize the reasoning model
reasoning_llm = AzureChatOpenAI(
    azure_endpoint= AZURE_OPENAI_ENDPOINT,
    azure_deployment=AZURE_OPENAI_DEPLOYMENT_O4,
    model=OPENAI_MODEL_O4,
    api_key=AZURE_OPENAI_API_KEY,
    api_version=OPENAI_API_VERSION,
    temperature = 1,
)

### Eksempel på ressonering vs. ikke-ressonering

In [119]:
# Example

prompt = PromptTemplate.from_template("""
A rope ladder hangs over the side of a boat. The rungs are 30 cm apart. The tide rises 1 meter every hour. After 3 hours, how many rungs will be underwater?
""")

reasoning_chain = prompt | reasoning_llm | StrOutputParser()

print("Svar fra resonneringsmodellen:", reasoning_chain.invoke({}))

none_reasoning_chain = prompt | llm | StrOutputParser()

print("\nSvar fra modellen uten resonnering:", none_reasoning_chain.invoke({}))

Svar fra resonneringsmodellen: None. As the tide rises, the boat (and its rope ladder) rises with it, so the number of submerged rungs never changes. If none were underwater to start, none will be after 3 hours.

Svar fra modellen uten resonnering: If the tide rises 1 meter every hour, after 3 hours, the tide will have risen:

\[
1 \text{ meter/hour} \times 3 \text{ hours} = 3 \text{ meters}
\]

Since the rungs of the ladder are 30 cm apart, we need to convert the rise in tide from meters to centimeters:

\[
3 \text{ meters} = 3 \times 100 \text{ cm} = 300 \text{ cm}
\]

Now, to find out how many rungs will be underwater, we divide the total rise in centimeters by the distance between the rungs:

\[
\frac{300 \text{ cm}}{30 \text{ cm/rung}} = 10 \text{ rungs}
\]

Therefore, after 3 hours, **10 rungs** will be underwater.


Aha! Her ser vi et eksempel på at resonneringsmodellen tenker riktig fordi den bryter ned problemet, mens det vanlige llm-kallet uten resonnering går fem på lureoppgaven og svarer feil.

# Nå lar vi modellen gå løs på hele datasettet

Nå skal vi bruke verktøyene vi har lært til å utføre den første oppgaven, nemlig å kategorisere tilbakemeldingene på tvers av datasettet. 

In [120]:
result_batch = categorize_chain_structured_output.batch(  # Pass all feedback to the model through batching
    [
    {"feedback": feedback} for feedback in df["Feedback"]
    ]
)

df["AI Classification"] = [result.category for result in result_batch]  # Add column containing the categories to the dataframe

In [86]:
df 

,ID,Feedback,AI Classification
0,1,"The network infrastructure is robust and usually efficient, though intermittent glitches sometimes hamper productivity and clarity.",Network
1,2,"Our current infrastructure aligns with business needs, yet occasional rigidity in legacy systems hinders innovative approaches.",Other
2,3,"I appreciate the proactive approach to security, though the multi-step authentication process sometimes seems overly complicated.",Other
3,4,"Our tools maintain high quality with intuitive design and regular updates, even though rare performance lags can hinder productivity.",Other
4,5,"The IT-support team is consistently prompt and courteous, though occasional miscommunications lead to fleeting moments of uncertainty.",IT-support
5,6,"Our advanced software tools offer a mix of efficiency and creativity, but the occasional downtime introduces a fleeting sense of dismay.",Network
6,7,"I feel more secure with the recent network defense upgrades, yet the complexity of the security software occasionally overwhelms my routine.",Network
7,8,"The new cybersecurity measures give me confidence in our data protection, though frequently changing protocols sometimes cause confusion.",Other
8,9,"Our enhanced firewall settings significantly lower threats, though frequent reminders to update can feel intrusive.",Other
9,10,"The assistance from IT-support is generally effective, though sporadic technical mishaps occasionally undermine steady progress.",IT-support


# Fot i bakken

Er konseptene vi har gått gjennom så langt tydelige, eller skal vi gå tilbake til noe før vi løser den siste oppgaven?

## Nye kategorier for 'Other'
Vi vil se nærmere på radene som falt under 'Other' og ikke ble kategorisert i første omgang. Til dette vil bruker vi en resonneringsmodell, men her er det lov å leke seg og sammenlikne med kategoriene en "vanlig" språkmodell finner frem til. 

Aller først sender vi alle tilbakemeldinger som havnet i "Other"-kategorien inn og ber modellen gi oss 3 nye kategorier den mener representerer dataen godt. 


In [123]:
categories_for_others_prompt = PromptTemplate.from_template(
"""
Give me a list of categories that best summarizes the following feedback. I need to be able to put each one of the feedbacks into a category.
Provide a maximum of 3 categories. Please provide clear, precise, and short categories with no '&' or 'and'.
 
Feedback:
<feedback>
{feedback}
</feedback>
"""
)

class Categories_for_others(BaseModel):
    "Categorization of a single feedback entry from an IT survey."
    categories : list[str] = Field(
        description="The best fitting category."
    )


categories_chain_others = categories_for_others_prompt | reasoning_llm.with_structured_output(
    Categories_for_others,
    method="json_schema",
    strict=True,
)
 
 
result = categories_chain_others.invoke(
    {"feedback": df[df['AI Classification'] == "Other"]['Feedback']}
)
 
NEW_CATEGORIES = Literal[tuple(result.categories)] # Converting the output from the model to a Literal

In [124]:
NEW_CATEGORIES

typing.Literal['Business alignment', 'Security protocols', 'System usability']

### Refleksjon
Se på innholdet i tilbakemeldingene over, synes du vi bør spørre etter flere/færre nye kategorier? Gjør endringer i prompten og se om du kan tvinge frem en enda bedre representasjon av innholdet. 

## Kategoriser resten av tilbakemeldingene
Nå vil vi bruke resulatet fra den forrige modellen til å kategorisere resten av feedbacken.

In [126]:

categorize_prompt_other = PromptTemplate.from_template(
"""
Categorize the following feedback from an IT-survey into the category that best describes the feedback.
 
Feedback:
<feedback>
{feedback}
</feedback>
"""
)

class Categorize_others(BaseModel):
    "Categorization of a single feedback entry from an IT survey."
    category : NEW_CATEGORIES = Field(
        description="The best fitting category. Only one."
    )
 
categorize_chain_others = categorize_prompt_other | reasoning_llm.with_structured_output(
    Categorize_others,
    method="json_schema",
    strict=True,
)
 
# Retrieve indexes for all rows where AI classification = "Other"
other_indices = df[df['AI Classification'] == "Other"].index
 
 
result_batch = categorize_chain_others.batch(
    [
    {"feedback": feedback} for feedback in df.loc[other_indices, "Feedback"]
    ]
)
 
df.loc[other_indices, "AI Classification"] = [result.category for result in result_batch]

In [127]:
df

,ID,Feedback,AI Classification
0,1,"The network infrastructure is robust and usually efficient, though intermittent glitches sometimes hamper productivity and clarity.",Network
1,2,"Our current infrastructure aligns with business needs, yet occasional rigidity in legacy systems hinders innovative approaches.",System usability
2,3,"I appreciate the proactive approach to security, though the multi-step authentication process sometimes seems overly complicated.",Security protocols
3,4,"Our tools maintain high quality with intuitive design and regular updates, even though rare performance lags can hinder productivity.",System usability
4,5,"The IT-support team is consistently prompt and courteous, though occasional miscommunications lead to fleeting moments of uncertainty.",IT-support
5,6,"Our advanced software tools offer a mix of efficiency and creativity, but the occasional downtime introduces a fleeting sense of dismay.",System usability
6,7,"I feel more secure with the recent network defense upgrades, yet the complexity of the security software occasionally overwhelms my routine.",Network
7,8,"The new cybersecurity measures give me confidence in our data protection, though frequently changing protocols sometimes cause confusion.",Security protocols
8,9,"Our enhanced firewall settings significantly lower threats, though frequent reminders to update can feel intrusive.",Security protocols
9,10,"The assistance from IT-support is generally effective, though sporadic technical mishaps occasionally undermine steady progress.",IT-support


Nå har vi identifisert passende kategorier for alle tilbakemeldingene. 
Videre kan vi få modellen til å oppsummere per kategori, slik at vi sitter igjen med en overordnet oversikt. 

## Hva nå?
Nå som kategoriseringen er gjennomført, og vi har forsikret oss om at alle radene har fått en passende kategori, kan vi gå videre til neste oppgave. Her er målet å samle informasjonen fra kategoriseringen til en overordnet oversikt. Vi kan enkelt lage en LCEL-kjede som tar resultatet fra forrige oppgave og oppsummerer per kategori ved hjelp av en LLM. Under ser dere et eksempel på hvordan: 

In [128]:
# 1. Prompt Template
summary_prompt = PromptTemplate.from_template("""
You are a domain expert in internal IT operations and organizational analysis. You will be provided with a dataset containing qualitative feedback from employees in an IT company. 
Each row in the dataset represents a feedback entry and is associated with a specific category.

For each category, carefully:
1. Read and interpret the feedback entries assigned to that category.
2. Identify core themes, recurring patterns, and contrasting opinions within that category.
3. Evaluate the feedback logically: What are the likely underlying causes of recurring issues or praises? Are there signs of systemic problems, isolated incidents, or misaligned expectations?
4. Summarize each category in 3 to 6 bullet points, highlighting key sentiments (positive and negative), representative concerns or compliments, and any significant outliers

Present your findings in a clean, professional way with one section per category. 

This is the employee feedback data: {survey_results}
""")

# 2. Class for structured output
class SummarizeFeedback(BaseModel):
    "Summary of the different categorizes recognized in the feedback from an IT-survey."
    summary : str = Field(
        description="For each category: Category name and 3-6 bullet points summarizing the category."
    )

# 3. Summary-chain
summary_chain_structured_output = summary_prompt | llm.with_structured_output(
    SummarizeFeedback,
    method="json_schema",
    strict=True,
)

# 4: Call the model with the dataset from the survey
summary = summary_chain_structured_output.invoke({"survey_results": df})

# A nice way to show the output from the model
display(Markdown(summary.summary))



### Network
- Generally robust and efficient, but intermittent glitches can hamper productivity.
- Occasional slowdowns during peak usage create anxiety about meeting deadlines.
- Strong connectivity is appreciated, though periodic drops raise concerns about data consistency.

### System Usability
- Tools are mostly user-friendly and intuitive, but some experience performance lags.
- Rapid changes in software can lead to confusion and misalignment with user needs.
- Occasional integration issues test patience, despite overall satisfaction with the tools.

### Security Protocols
- Proactive security measures instill confidence, but complexity can overwhelm users.
- Frequent updates and alerts can feel intrusive, leading to slight irritation.
- Robust protocols are generally effective, though sporadic false alarms create unnecessary concerns.

### IT-Support
- IT-support is often prompt and courteous, providing effective resolutions.
- Communication gaps and backlog delays occasionally disrupt workflow and create apprehension.
- Overall, support is seen as engaging and resourceful, but some recurring issues lead to frustration.

### Training
- Training sessions are generally beneficial, though pacing can lead to uneven clarity.
- Hands-on exercises are engaging, but some struggle to keep up with the fast pace.
- Content depth sometimes exceeds comfort levels, leaving parts underexplored.

### Business Alignment
- IT solutions are mostly aligned with business needs, but occasional mismatches create operational bottlenecks.
- Efforts to merge IT and business strategies are recognized, though clarity on direction is sometimes lacking.

## Fantastisk!
Du har nå fullført kurset LLM for alle, og jobbet deg gjennom hvordan du kan koble deg til språkmodeller via API-er, hvordan du kan skrive LCEL for å lage gjenbrukbare prompt-maler og AI-kjeder, hvordan du kan sikre presise svar med structured output og sett på oppgaver der det kan være nyttig å bruke en resonneringsmodell fremfor en vanlig språkmodell.

# Til ettertanke


### *Hvordan kunne vi gjort dette bedre?*

Anta at du har fått levert resultatene fra denne spørreundersøkelsen i fanget av en stressa mellomleder som ber deg levere en rapport han kan presentere ledelsen. 
Gitt verktøyene du har fått en innføring i gjennom dette kursene (og kanskje andre erfaringer?), hvordan ville du løst oppgaven?

Ser du for eksempel noe som kunne vært forbedret i
- Rekkefølgen på måten vi leter etter kategorier?
- Legger vi får mye/lite vekt på inputen vi fikk om hva ledelsen "tror" kategoriene kommer til å være?
- Er vi for "strenge" når vi ber modellen finne kategorier? Kan man gjøre endringer for å plukke opp flere nyanser?
- Promptingen?
- Variablene eller type hintingen i pydantic-klassene?

Ville du kanskje gjort det helt annerledes? 
Dette er sprøsmål det kan lønne seg å gruble litt over i etterkant av kurset, kanskje vil du også prøve deg frem litt på egenhånd? Vi er bare glade for å svare på eventuelle spørsmål dere har på stand etterpå, eller på linkedin eller over en kaffe ved en senere anledning! 

### *Hva er risikoene ved bruk av LLM til denne typen analyse?*
Ved bruk av store språkmodeller (LLM) til analyse av åpne tekstsvar i spørreundersøkelser er risikoen generelt lavere enn i mer kritiske anvendelser som faktagenerering eller beslutningsstøtte. Likevel finnes det enkelte utfordringer som bør tas på alvor for å sikre kvalitet og transparens i analysen.
 
#### Skjevheter og bias
LLM-er kan tolke og vekte visse formuleringer eller temaer ulikt basert på mønstre fra treningsdataene. Dette kan føre til at noen svar over- eller underrepresenteres i visse kategorier avhengig av hvordan modellen "forstår" teksten. For eksempel kan emosjonelt ladede eller språklig avanserte svar få uforholdsmessig stor plass, mens mer nøkterne eller utypiske svar blir feilklassifisert. Særlig ved tolkning av følsomme temaer er det viktig å være oppmerksom på disse tendensene.
 
#### Hallusinasjon – mindre relevant, men ikke fraværende
I denne typen oppgave, hvor modellen hovedsakelig skal kategorisere eller oppsummere eksisterende tekstsvar er risikoen for såkalte hallusinasjoner (altså at modellen «finner opp» informasjon) mindre, men ikke helt ubetydelig. Det kan forekomme at modellen overfortolker et kort eller tvetydig svar og tillegger det en mening som ikke eksplisitt finnes i teksten. Dette kan føre til at enkelte svar havner i feil kategori eller påvirker tolkningen av sentiment eller tematikk.
 
#### Behov for kvalitetssikring
Selv med høy presisjon og effektivitet bør man ikke ta modellens resultater for gitt. Det anbefales å gjennomføre stikkprøver, sammenligne med manuell koding og evaluere nøyaktigheten på klassifiseringene. Det kan være veldig nyttig å kombinere automatisk kategorisering med menneskelig vurdering både i startfasen og underveis.